<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 4 / SPRINT 2 / NOTE 2*

# 📝 Assignment

---


# Distributed_Representation

## Code

캐글의 [SMS Spam dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset) 에 사전 학습된 Word2Vec 임베딩 벡터를 적용하여 분류해봅시다.<br/>
세션 노트에 있었던 단어 임베딩 벡터를 평균내어 분류하는 방법을 적용해봅시다.

In [1]:
!pip install gensim --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
import gensim

gensim.__version__

'4.2.0'

In [3]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


### 1. Word2Vec과 코사인 유사도

word2vec을 이용해 구한 'data'와 'science'임베딩 값의 코사인 유사도를 구해보겠습니다. sklearn의 cosine_similarity를 이용하겠습니다.

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(wv['data'].reshape(1, -1), wv['science'].reshape(1, -1))

array([[0.1575913]], dtype=float32)

문항 1) 위에서 구한 코사인 유사도를 소수점 3째자리까지 입력하세요

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer

### 2. 텍스트 분류

### 1) 데이터 전처리
    
- 데이터셋을 데이터프레임으로 읽어옵니다 `encoding = 'latin-1'` 을 사용합니다.
- 필요없는 열(column)을 삭제합니다.
- LabelEncoder를 사용하여 label 전처리를 해줍니다. 

In [7]:
from google.colab import files

file = files.upload()

Saving spam.csv to spam.csv


In [13]:
df = pd.read_csv('spam.csv', encoding='latin-1').dropna(axis= 1)

In [14]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [15]:
df['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [16]:
df['v1'] = LabelEncoder().fit_transform(df['v1'])
df

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


### 2) 텍스트 분류를 수행해주세요.

- 데이터셋 split시 test_size의 비율은 15%로, `random_state = 42` 로 설정합니다. 
- Tokenizer의 `num_words = 1000` 으로 설정합니다.
- pad_sequence의 `maxlen=150` 으로 설정합니다.
- 학습 시, 파라미터는 `batch_size=64, epochs=10, validation_split=0.2` 로 설정합니다.
- evaluate 했을 때의 loss와 accuarcy를 [loss, acc] 형태로 입력해주세요. Ex) [0.4321, 0.8765]

In [11]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
### 이곳에서 과제를 수행해 주세요 ###
X_train, X_test, y_train, y_test = train_test_split(
    df['v2'], df['v1'], test_size=0.15, random_state= 42)

tokenizer = Tokenizer(num_words= 1000)
tokenizer.fit_on_texts(X_train)
X_train_encoded = tokenizer.texts_to_sequences(X_train)
X_test_encoded  = tokenizer.texts_to_sequences(X_test)

X_train_encoded

In [18]:
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size, 300))
print(vocab_size)

8210


In [19]:
def get_vector(word):
    """
    해당 word가 word2vec에 있는 단어일 경우 임베딩 벡터를 반환
    """
    if word in wv:
        return wv[word]
    else:
        return None

for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp


In [21]:
X_train_pad = pad_sequences(X_train_encoded, maxlen=150)
X_test_pad  = pad_sequences(X_test_encoded,  maxlen=150)

model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=150, trainable=False))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', metrics= ['acc'])
model.fit(X_train_pad, y_train, batch_size=64, epochs=10, validation_split=0.2)
model.evaluate(X_test_pad, y_test)

Epoch 1/10
60/60 [==============================] - 2s 12ms/step - loss: 0.6659 - acc: 0.8648 - val_loss: 0.6455 - val_acc: 0.8660
Epoch 2/10
60/60 [==============================] - 1s 9ms/step - loss: 0.6292 - acc: 0.8648 - val_loss: 0.6109 - val_acc: 0.8660
Epoch 3/10
60/60 [==============================] - 1s 9ms/step - loss: 0.5971 - acc: 0.8648 - val_loss: 0.5800 - val_acc: 0.8660
Epoch 4/10
60/60 [==============================] - 1s 10ms/step - loss: 0.5690 - acc: 0.8648 - val_loss: 0.5531 - val_acc: 0.8660
Epoch 5/10
60/60 [==============================] - 1s 9ms/step - loss: 0.5438 - acc: 0.8648 - val_loss: 0.5289 - val_acc: 0.8660
Epoch 6/10
60/60 [==============================] - 1s 10ms/step - loss: 0.5214 - acc: 0.8648 - val_loss: 0.5076 - val_acc: 0.8660
Epoch 7/10
60/60 [==============================] - 1s 9ms/step - loss: 0.5021 - acc: 0.8648 - val_loss: 0.4897 - val_acc: 0.8660
Epoch 8/10
60/60 [==============================] - 1s 10ms/step - loss: 0.4856 - acc: 

[0.44735926389694214, 0.8708133697509766]

### 3)Word2Vec에서의 OOV 문제

```
def get_vector(word):
    """
    해당 word가 word2vec에 있는 단어일 경우 임베딩 벡터를 반환
    """
    if word in wv:
        return wv[word]
    else:
        return None
 
for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp
```
Lecture Note에 있는 위의 코드를 변형하여, OOV 개수를 확인해주세요.
- tokenizer는 위에서 활용한 tokenizer를 그대로 사용하겠습니다.
- Tip : dictionary를 활용하거나, Counter를 활용해보세요.

In [32]:
### 이곳에서 과제를 수행해 주세요 ###
oov = []
for word, _ in tokenizer.word_index.items():
  if word not in wv:
    oov.append(word)

print(len(oov))

2419


In [30]:
tokenizer.word_index.items()

dict_items([('i', 1), ('to', 2), ('you', 3), ('a', 4), ('the', 5), ('u', 6), ('and', 7), ('in', 8), ('is', 9), ('me', 10), ('my', 11), ('for', 12), ('your', 13), ('it', 14), ('of', 15), ('call', 16), ('have', 17), ('on', 18), ('2', 19), ('that', 20), ('now', 21), ('are', 22), ('so', 23), ('but', 24), ('not', 25), ('or', 26), ('at', 27), ("i'm", 28), ('do', 29), ('ur', 30), ('can', 31), ('get', 32), ('will', 33), ('if', 34), ('be', 35), ('just', 36), ('with', 37), ('no', 38), ('we', 39), ('gt', 40), ('this', 41), ('lt', 42), ('up', 43), ('4', 44), ('when', 45), ('from', 46), ('ok', 47), ('free', 48), ('go', 49), ('how', 50), ('what', 51), ('all', 52), ('out', 53), ('know', 54), ('got', 55), ('good', 56), ('like', 57), ('then', 58), ('was', 59), ('come', 60), ('its', 61), ('am', 62), ('time', 63), ('only', 64), ('day', 65), ('send', 66), ('love', 67), ('there', 68), ('text', 69), ('he', 70), ('want', 71), ('txt', 72), ('going', 73), ('home', 74), ('one', 75), ('about', 76), ('as', 77), (